In [1]:
!pip install xdggs
!pip install healpy
!pip install -U zarr==2.18.4
!pip install numcodecs==0.13.1


  Using cached xdggs-0.1.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached cdshealpix-0.7.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached h3ronpy-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.8 kB)
  Using cached arro3_core-0.4.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (885 bytes)
Using cached xdggs-0.1.1-py3-none-any.whl (32 kB)
Using cached arro3_core-0.4.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
Using cached cdshealpix-0.7.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
Using cached h3ronpy-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)
  Using cached healpy-1.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.7 kB)
Using cached healpy-1.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.9 MB)
  Using cached zarr-2.18.4-py3-none-any.whl.metadata (5.8 kB)
Using cac

In [4]:
!pip install -U xarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2024.7.0
    Uninstalling xarray-2024.7.0:
      Successfully uninstalled xarray-2024.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmip 0.7.2 requires xgcm<0.7.0, but you have xgcm 0.8.1 which is incompatible.


In [1]:
import warnings
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xdggs
import fsspec

warnings.filterwarnings("ignore")
xr.set_options(display_max_rows=27)

In [7]:
#PB, opening dataree here. why?

fs = fsspec.filesystem("http")
url="/home/jovyan/Akathon-Geilo/odysea_8_nest.zarr"
url="https://data-fair2adapt.ifremer.fr/odysea/odysea.zarr"
url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_all_nest.zarr"
#https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_10_nest.zarr/
dtotal=xr.open_datatree(fs.open(url), engine="zarr")#,engine='zarr')
#.pipe(xdggs.decode)
#dtotal=xr.open_zarr().pipe(xdggs.decode)
dtotal

ValueError: Starting with Zarr 2.11.0, stores must be subclasses of BaseStore, if your store exposes the MutableMapping interface wrap it in Zarr.storage.KVStore. Got <File-like object HTTPFileSystem, https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_all_nest.zarr>

In [8]:
import numpy as np


def create_arrow_table(polygons, arr, coords=None):
    from arro3.core import Array, ChunkedArray, Schema, Table

    if coords is None:
        coords = ["latitude", "longitude"]

    array = Array.from_arrow(polygons)
    name = arr.name or "data"
    arrow_arrays = {
        "geometry": array,
        "cell_ids": ChunkedArray([Array.from_numpy(arr.coords["cell_ids"])]),
        name: ChunkedArray([Array.from_numpy(arr.data)]),
    } | {
        coord: ChunkedArray([Array.from_numpy(arr.coords[coord].data)])
        for coord in coords
        if coord in arr.coords
    }

    fields = [array.field.with_name(name) for name, array in arrow_arrays.items()]
    schema = Schema(fields)

    return Table.from_arrays(list(arrow_arrays.values()), schema=schema)


def normalize(var, center=None):
    from matplotlib.colors import CenteredNorm, Normalize

    if center is None:
        vmin = var.min(skipna=True)
        vmax = var.max(skipna=True)
        normalizer = Normalize(vmin=vmin, vmax=vmax)
    else:
        halfrange = np.abs(var - center).max(skipna=True)
        normalizer = CenteredNorm(vcenter=center, halfrange=halfrange)

    return normalizer(var.data)


def exploire_layer(
    arr,
    cell_dim="cells",
    cmap="viridis",
    center=None,
    alpha=None,
):
    from lonboard import SolidPolygonLayer
    from lonboard.colormap import apply_continuous_cmap
    from matplotlib import colormaps

    if len(arr.dims) != 1 or cell_dim not in arr.dims:
        raise ValueError(
            f"exploration only works with a single dimension ('{cell_dim}')"
        )

    cell_ids = arr.dggs.coord.data
    grid_info = arr.dggs.grid_info

    polygons = grid_info.cell_boundaries(cell_ids, backend="geoarrow")

    normalized_data = normalize(arr.variable, center=center)

    colormap = colormaps[cmap]
    colors = apply_continuous_cmap(normalized_data, colormap, alpha=alpha)

    table = create_arrow_table(polygons, arr)
    layer = SolidPolygonLayer(table=table, filled=True, get_fill_color=colors)

    return layer

In [9]:
url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_9_nest.zarr"

xr.open_zarr( url)

<xarray.Dataset> Size: 19GB
Dimensions:               (cells: 3145728, orbit: 29)
Coordinates:
    cell_ids              (cells) int64 25MB dask.array<chunksize=(1048576,), meta=np.ndarray>
  * orbit                 (orbit) int64 232B 161 162 163 164 ... 186 187 188 189
Dimensions without coordinates: cells
Data variables:
    azimuth_aft           (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    azimuth_fore          (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    encoder_aft           (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    encoder_fore          (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    hit                   (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    lat                   (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    lon                   (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    radial_angle_aft      (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    radial_angle_fore     (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    sample_time           (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    u_model               (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_ac                 (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_aft                (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_al                 (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_eastward           (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_fore               (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_nonoise_ac         (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_nonoise_aft        (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_nonoise_al         (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_nonoise_eastward   (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_nonoise_fore       (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_nonoise_northward  (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    ur_northward          (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    v_model               (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    wind_dir              (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    wind_speed            (cells, orbit) float64 730MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>

In [24]:
import lonboard
url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_9_nest.zarr"
arr9 = (
    xr.open_zarr(url)[["u_model"]]
    .pipe(xdggs.decode)  # .dggs.sel_latlon(latitude=48, longitude=0)["data"]
)
ds=np.sqrt(dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2+dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2).dggs.sel_latlon(longitude=full_lon, latitude=full_lat).mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)

url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_10_nest.zarr"
arr10 = (
    xr.open_zarr(url)[["u_model"]].isel(orbit=0)
    .pipe(xdggs.decode)  # .dggs.sel_latlon(latitude=49, longitude=0)["data"]
)
# cmap="Reds"
lonboard.Map(
    [
        exploire_layer(
            arr9,
            alpha=1,
        ),
        exploire_layer(arr10, alpha=1),
    ]
)

ValueError: no DGGSIndex found on this Dataset or DataArray

In [23]:
url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_9_nest.zarr"

dtotal=xr.open_zarr(url)
dtotal
ds=np.sqrt(dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2+dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2).dggs.sel_latlon(longitude=full_lon, latitude=full_lat).mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
ds

ValueError: no DGGSIndex found on this Dataset or DataArray

In [18]:
url="https://data-fair2adapt.ifremer.fr/odysea/test/F_c13_9_nest.zarr"
dtotal = (
    xr.open_zarr(url)[["u_model","v_model"]]#.isel(orbit=0)
    .pipe(xdggs.decode)  # .dggs.sel_latlon(latitude=48, longitude=0)["data"]
)
dtotal

<xarray.Dataset> Size: 2GB
Dimensions:   (cells: 3145728, orbit: 39)
Coordinates:
  * cell_ids  (cells) int64 25MB dask.array<chunksize=(1048576,), meta=np.ndarray>
  * orbit     (orbit) int64 312B 161 162 163 164 165 166 ... 195 196 197 198 199
Dimensions without coordinates: cells
Data variables:
    u_model   (cells, orbit) float64 981MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
    v_model   (cells, orbit) float64 981MB dask.array<chunksize=(1048576, 1), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=9, indexing_scheme=nested)

In [19]:

lat = np.arange(40, 70, 0.03)
lon = np.arange(-24, 15, 0.03)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))
ds=np.sqrt(dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2+dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2).dggs.sel_latlon(longitude=full_lon, latitude=full_lat).mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
ds.dggs.explore(alpha=0.8,cmap='coolwarm')
#dtotal.u_model.where(lambda x : x!=hp.UNSEEN).mean("orbit").compute().dggs.explore(cmap='jet',alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7b61b5c2…

In [24]:
dtotal=xr.open_zarr("/home/jovyan/Akathon-Geilo/odysea_9_nest.zarr").pipe(xdggs.decode)
#dtotal=xr.open_zarr("https://data-fair2adapt.ifremer.fr/odysea/odysea.zarr").pipe(xdggs.decode)
dtotal

<xarray.Dataset> Size: 1GB
Dimensions:               (cells: 3145728, orbit: 2)
Coordinates:
  * cell_ids              (cells) int64 25MB dask.array<chunksize=(3145728,), meta=np.ndarray>
  * orbit                 (orbit) int64 16B 161 162
Dimensions without coordinates: cells
Data variables:
    azimuth_aft           (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    azimuth_fore          (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    encoder_aft           (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    encoder_fore          (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    hit                   (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    lat                   (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    lon                   (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    radial_angle_aft      (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    radial_angle_fore     (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    sample_time           (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    u_model               (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_ac                 (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_aft                (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_al                 (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_eastward           (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_fore               (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_nonoise_ac         (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_nonoise_aft        (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_nonoise_al         (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_nonoise_eastward   (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_nonoise_fore       (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_nonoise_northward  (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    ur_northward          (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    v_model               (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    wind_dir              (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
    wind_speed            (cells, orbit) float64 50MB dask.array<chunksize=(3145728, 1), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=9, indexing_scheme=nested)

In [25]:
lat = np.arange(40, 70, 0.03)
lon = np.arange(-24, 15, 0.03)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))
ds=np.sqrt(dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2+dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2).dggs.sel_latlon(longitude=full_lon, latitude=full_lat).mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
ds.dggs.explore(alpha=0.8,cmap='coolwarm')
#dtotal.u_model.where(lambda x : x!=hp.UNSEEN).mean("orbit").compute().dggs.explore(cmap='jet',alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x78bb7acc…

In [26]:
dtotal=xr.open_zarr("/home/jovyan/Akathon-Geilo/odysea_10_nest.zarr").pipe(xdggs.decode)
#dtotal=xr.open_zarr("https://data-fair2adapt.ifremer.fr/odysea/odysea.zarr").pipe(xdggs.decode)
dtotal

<xarray.Dataset> Size: 5GB
Dimensions:               (cells: 12582912, orbit: 2)
Coordinates:
  * cell_ids              (cells) int64 101MB dask.array<chunksize=(4194304,), meta=np.ndarray>
  * orbit                 (orbit) int64 16B 161 162
Dimensions without coordinates: cells
Data variables:
    azimuth_aft           (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    azimuth_fore          (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    encoder_aft           (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    encoder_fore          (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    hit                   (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    lat                   (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    lon                   (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    radial_angle_aft      (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    radial_angle_fore     (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    sample_time           (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    u_model               (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_ac                 (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_aft                (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_al                 (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_eastward           (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_fore               (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_ac         (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_aft        (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_al         (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_eastward   (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_fore       (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_northward  (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_northward          (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    v_model               (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    wind_dir              (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    wind_speed            (cells, orbit) float64 201MB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=10, indexing_scheme=nested)

In [27]:
lat = np.arange(40, 70, 0.03)
lon = np.arange(-24, 15, 0.03)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))
ds=np.sqrt(dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2+dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2).dggs.sel_latlon(longitude=full_lon, latitude=full_lat).mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
ds.dggs.explore(alpha=0.8,cmap='coolwarm')
#dtotal.u_model.where(lambda x : x!=hp.UNSEEN).mean("orbit").compute().dggs.explore(cmap='jet',alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x78bb7ac0…